## To-do:
1. Adjust the code for the clean "DOF" issue
2. Fix the issue -- adjust the isues with the at_risk date
2. Fix the "Free Time Issue"
3. Make sure that the final exported dataframe is at the id_variable, dos level
4. Document (in Word Doc or even at the end of this file the logic used at each step of the process and why decisions where made)

## Ongoing Questions:
insert any lingering questions here
1. Should we subset the data to where dof < OR equal to dos or JUST keep data that is strictly less then (dof < dos)

## Load Data

In [58]:
#this jupyter notebook is essentially the same as the "recidivism-check" notebook, just cleaned up a bit (hence the name)
#import required libraries
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import sqlite3

#get the folder path for this data
pa_sentencing_path = os.path.dirname(os.path.dirname(os.path.dirname(os.getcwd())))

#read in the correct data file (need to read in this file because of the additional columns it has)
psc_trimmed = pd.read_csv(os.path.join(pa_sentencing_path, "Project", "data", "PSC_data_trimmed_v1.csv"))

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (19,29,45,46,48,49,50,51,52,53,59,63,64,66) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## Run this code chunk below only once 

In [59]:
## Run this only once to create an in-memory database for the data (should speed up retrieval times)

#read in the trimmed dataset
# psc_trimmed = pd.read_csv(os.path.join(pa_sentencing_path, "Project", "data", "Main.csv"),
# usecols = ['JPR_ID','id_variable','dof','dos','prs','INC_SANCTION_EXISTS','JP_CC_BUG','JP_MIN','JP_LIFE_DEATH','OFN_LIFE_DEATH'])

# conn = sqlite3.connect("psc_data2.db")
# cur = conn.cursor()
# ## Creating Table
# cur.execute('''CREATE TABLE IF NOT EXISTS raw_data  
#         (JPR_ID	INTEGER, 
#         prs TEXT, 
#         INC_SANCTION_EXISTS TEXT, 
#         dof TEXT,	
#         dos TEXT, 
#         JP_MIN TEXT,	
#         OFN_LIFE_DEATH TEXT,
#         id_variable INTEGER, 
#         JP_LIFE_DEATH TEXT,
#         JP_CC_BUG TEXT)''')

# insert_query = """ INSERT INTO raw_data (
#    JPR_ID,	
#    prs,	
#    INC_SANCTION_EXISTS,
#    dof,	
#    dos,	
#    JP_MIN,	
#    OFN_LIFE_DEATH,	
#    id_variable,	
#    JP_LIFE_DEATH,	
#    JP_CC_BUG)
#    VALUES (?,?,?,?,?,?,?,?,?,?)"""
# raw_list = psc_trimmed.values.tolist()    
# cur.executemany(insert_query, raw_list)
# conn.commit()

In [60]:
# import os
# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt
# import seaborn as sns
# import datetime
# import sqlite3
# def load_data():
#     """ Read the sqlite database, from the file ".db" into a pandas dataframe
#     Returns:
#         pd.DataFrame : a dataframe 
#     """    
#     conn = sqlite3.connect("psc_data2.db")
#     df_raw = pd.read_sql_query("SELECT * FROM raw_data", conn)
#     return(df_raw)

# df_tbl_db = load_data()

In [61]:
# copying the original loaded data to a working data frame to use and compare with later
#df = df_tbl_db.copy() #if accessing the database

df = psc_trimmed.copy() # if accessing the psc_trimmed file directly


#change column names to uppercase
df.columns = df.columns.str.upper()


In [62]:
df.head() #inspect the dataset

,JPR_ID,OFF_SEX,OFF_RACE,DOFAGE,OTN,OFN_TITLE,OFN_COUNT,OFN_LABEL,OFN_GRADE,GRADE,...,STAT_MIN,DISPOSITION,CONFORMITY,REASON_ONE,REASON_TWO,REASON_THREE,MORE_REASONS,PRS_MANUAL,PRS_LAPSING,PRS_NONLAPSING
0,640001,F,White,36.689938,H182628-5,18,1,Corruption of Minors - when of a sexual nature,M-1,4,...,30,Nolo Contendere,Standard,NaN,NaN,NaN,False,NaN,0,0
1,642480,M,White,18.540726,0,75,1,DUI - M-2,M-2,3,...,12,Neg. Guilty Plea,Standard/Mandatory,NaN,NaN,NaN,False,NaN,0,0
2,660434,M,White,36.914442,H3618344,75,1,DUI - M-2,M-2,3,...,12,Non-Neg. Guilty Plea,Standard/Mandatory,NaN,NaN,NaN,False,NaN,0,0
3,628940,M,Black,22.297057,G0816126,18,1,Simple Assault,M-2,3,...,12,Neg. Guilty Plea,Standard,NaN,NaN,NaN,False,NaN,0,0
4,594048,M,White,40.087611,H240127-6,75,1,DUI - M-1,M-1,4,...,30,NaN,Standard/Mandatory,NaN,NaN,NaN,False,NaN,1,1


## Convert Dates

In [63]:
#convert date strings to datetime variable
df[['DOF','DOS']] = df[['DOF','DOS']].apply(pd.to_datetime,format="%d %b %y")


In [64]:
# extracting out the just the year from the date to be used later 
df['DOF_YEAR'] = pd.DatetimeIndex(df['DOF']).year
df['DOS_YEAR'] = pd.DatetimeIndex(df['DOS']).year

In [65]:
#checking the range of values for the DOF and DOS variables
print("The minimum date of offense in the dataset is: {}".format(df[["DOF"]].min()[0]))
print("The maximum date of offense in the dataset is: {}".format(df[["DOF"]].max()[0]))
print("The minimum date of sentencing in the dataset is: {}".format(df[["DOS"]].min()[0]))
print("The maximum date of sentencing in the dataset is: {}".format(df[["DOS"]].max()[0]))

The minimum date of offense in the dataset is: 1984-11-14 00:00:00
The maximum date of offense in the dataset is: 2020-05-08 00:00:00
The minimum date of sentencing in the dataset is: 2001-01-01 00:00:00
The maximum date of sentencing in the dataset is: 2019-12-31 00:00:00


##### Note: As shown in the above code chunk, there **isn't** anamolous behavior in the date ranges (i.e. a date in the year 1909 or 2090) for the date of offense (DOF) or date of sentence (DOS) variables -- therefore, an additional date correction was **not** applied in this case.

## Clean DOS > DOF

Note: group offense by ID_VAR, JPR_ID, MIN(DOF) to get the first DOF associated for a single JPR_ID

In [66]:
# df_no_missing = df[df['DOF'].notnull()]

# #subset to just those rows where DOF is missing
# dof_missing = df[df['DOF'].isnull()]
# not needed

#df[:20].head() 

#count how many values of DOF are missing in the original dataset
dof_missing = df[df['DOF'].isnull()]


print("There are {:,} rows with missing DOFs in the dataset.".format(len(dof_missing)))



There are 15,965 rows with missing DOFs in the dataset.


In [67]:
#group offense by ID_VAR, JPR_ID, MIN(DOF) to get the first DOF associated for a single JPR_ID -- fix issue with missing DOF
# for name, group in df.groupby("JPR_ID"):
#     #only applies if there is more than one charge 

#     #number of charges in a group
#     num_chargs = len(group)

#     if num_chargs > 1:
#         list_of_dof = group['DOF'].tolist()
#         min_dof = min(list_of_dof)

#         #check if nas are in the list and ONLY change the DOF to MIN IF the DOF is missing
#         updated_dof  = []
#         for x in list_of_dof:
#             if pd.isnull(x):
#                 updated_dof.append(min_dof)
#             else:
#                 updated_dof.append(x)
  
#         #only reassign IF for that row the DOF is missing
#         df.loc[df["JPR_ID"] == name, 'DOF'] = updated_dof

#at the JPR_ID level we only want ONE DOF because becuase we don't want to take into account DOF's that occur
#BEFORE the DOS (associated with the JPR_ID) as an instance of recidivism. -- each JPR_ID should have only ONE DOS
# for name, group in df[:20].groupby("JPR_ID"):  
#     #only applies if there is more than one charge 

#     #number of charges in a group
#     num_chargs = len(group)

#     if num_chargs > 1:
#         list_of_dof = group['DOF'].tolist()
#         min_dof = min(list_of_dof)
#         print(list_of_dof)
#         print(min_dof)

#         updated_dof = [min_dof for x in list_of_dof]

#         #reassign the DOF to the minimum for that JPR_ID
#         df.loc[df["JPR_ID"] == name, 'DOF'] = updated_dof


# df[:20].head()[["JPR_ID", "DOF"]]


In [68]:
#subset the data just to where the number of charges for a given JPR_ID is > 1
# counts_by_jpr_id = df.groupby(["JPR_ID"]).agg({"DOF": "count"})

# multiple_chargs_jpr_ids = list(counts_by_jpr_id.loc[counts_by_jpr_id["DOF"] > 1].index)

# #print(len(multiple_chargs_jpr_ids))

# test_df = df[df["JPR_ID"].isin(multiple_chargs_jpr_ids)]

#print(len(test_df))

In [69]:
#test_df.head()[["JPR_ID", "DOF"]]

#want the earliest offense for a given JPR_ID

In [70]:
#at the JPR_ID level we only want ONE DOF because becuase we don't want to take into account DOF's that occur
#BEFORE the DOS (associated with the JPR_ID) as an instance of recidivism. -- each JPR_ID should have only ONE DOS

#test_df["NEW_DOF"] = test_df.groupby(["JPR_ID"])["DOF"].transform("min")


#test_df.head()[["JPR_ID", "DOF", "NEW_DOF"]]
#one_test_example = test_df.loc[test_df["JPR_ID"] == 658826][["DOF", "NEW_DOF"]] 
# one_test_example["NEW_DOF"] = one_test_example.groupby(["JPR_ID"])["DOF"].transform("min")
# one_test_example


### **Step 1**: Make sure that we are only looking at the **minimum** value for the DOF across all of the charges associated with **one** JPR_ID. This is the procedure because we don't wan't to count a DOF as an instance of recidivism if it occurs BEFORE the date of sentencing

In [71]:
#at the JPR_ID level we only want ONE DOF because becuase we don't want to take into account DOF's that occur
#BEFORE the DOS (associated with the JPR_ID) as an instance of recidivism. -- each JPR_ID should have only ONE DOS

df["NEW_DOF"] = df.groupby(["JPR_ID"])["DOF"].transform("min")

In [72]:
df.head()[["JPR_ID", "DOF", "NEW_DOF"]]

,JPR_ID,DOF,NEW_DOF
0,640001,2000-04-01,2000-04-01
1,642480,1999-12-31,1999-12-31
2,660434,2000-12-23,2000-12-23
3,628940,2000-06-26,2000-06-26
4,594048,2000-10-15,2000-10-15


In [73]:
dof_missing = df[df['NEW_DOF'].isnull()]

percent_missing = len(dof_missing)/len(df)
print("After cleaning, there are {:,} ({:%}) rows with missing DOFs in the dataset.".format(len(dof_missing), percent_missing))



#random testing code here
#df.head()
# test = [pd.NaT, 1,2, 3]
# updated_dof = [x for x in test if pd.isnull(x)] 
# updated_dof

After cleaning, there are 11,785 (0.454381%) rows with missing DOFs in the dataset.


### **Step 2**: Subset the data to just include those rows where NEW_DOF <= DOS

In [74]:
#make sure the sentencing 
before_length = len(df)
df = df[df.NEW_DOF <= df.DOS] #should this be <= ?
after_length = len(df)

print("Before DOF <= DOS correction there were {:,} rows and after cleaning there were {:,} rows. A change of {:,}.".format(before_length, after_length, before_length - after_length))


Before DOF <= DOS correction there were 2,593,636 rows and after cleaning there were 2,581,813 rows. A change of 11,823.


In [75]:
#print(len(psc_trimmed), len(df), len(psc_trimmed) - len(df))

2593636 2581813 11823


## Clean Missing PRS Score 

In [ ]:
#subset to just the id variables with a PRS score missing
# id_varswith_prsmissing= set(df[df.prs.isnull()].id_variable)
# #remove id vars with missing PRS
# df_prs_notaffected = df[~df.id_variable.isin(id_varswith_prsmissing)]
# #reassign to working dataframe
# df = df_prs_notaffected 

In [ ]:
# PRS SCORE CLEANING - removes only the rows where the PRS score is missing not the entire individual 
before_length = len(df)
df = df.loc[df['PRS'].notnull()]
after_length = len(df)

print("Before PRS correction there were {:,} rows and after cleaning there were {:,} rows. A change of {:,}.".format(before_length, after_length, before_length - after_length))




## Clean JP CC Bug

In [ ]:
# Obtaining the id variables with jp_bug
id_varswith_jpbug= set(df[df.JP_CC_BUG=='Y'].id_variable)

In [ ]:
# assigning all the rows associated with the jp bugs to a seperate dataframe 
df_with_jpbug=  df[df.id_variable.isin(id_varswith_jpbug)]

In [ ]:
# Removing the rows of these that are beyond 2016(rows in the future that are affected by the JP_CC_BUG)
df_jp_bug_cleaned = df_with_jpbug[df.dos_year<2016]

In [ ]:
# Isolating the rows associated with id_vars in the original dataframe that is not associated with the bug
df_jpbug_notaffected = df[~df.id_variable.isin(id_varswith_jpbug)]

In [ ]:
# Rejoining the rows affected by the JP_CC_bug after cleaning them to the rows not affected by the bug
df_cleaned_1 = pd.concat([df_jpbug_notaffected,df_jp_bug_cleaned])  #new wor

df = df_cleaned_1

## Implement At Risk Date Calculation Logic

NEED TO ADD THIS TO TAKE CARE OF TIME SERVED AND POTENTIALLY DIFFERENT JP_MIN VALS

IF INC_SANCTION_EXISTS = ‘N’: do nothing 
ELIF INC_SANCTION_EXISTS on at least 1 charge:
If DOS is the same across all charges:
Get MAX(JP_MIN) across charges for this JPR_ID
If DOS is different across charges:
Use JP_MIN + MAX(DOS) - [ MAX(DOS) - MIN(DOS) ] to get time incarcerated 
Should this be: JP_MIN + (MAX(DOS) + [ MAX(DOS) - MIN(DOS) ])


In [ ]:
def create_at_risk_date(row):
    #need to account for REALLY large JP_MIN values
    #20 years in days = 10 * 365
    upper_limit = 20.0 * 365.0
    
    num_days_in_month = 30.0

    if row['OFN_LIFE_DEATH'] == "Y":
        at_risk_date = pd.to_datetime('9999-12-31')
    
    else:

        if row["JP_MIN"] < upper_limit:

            if row["INC_SANCTION_EXISTS"] == "Y" and pd.notna(row['JP_MIN']):
                at_risk_date = row['DOS'] + pd.Timedelta(days = row['JP_MIN'])
            
            elif row["INC_SANCTION_EXISTS"] == "Y" and pd.notna(row['INCMIN']):
                at_risk_date = row['DOS'] + pd.Timedelta(days = row['INCMIN'] * num_days_in_month)

            #these are individuals who have life and death sentences
            #their inc_end date = 12/31/9999 -- see codebook for more details
            elif row["INC_SANCTION_EXISTS"] == "Y" and pd.isna(row['INCMIN']) and pd.isna(row['JP_MIN']) :
                at_risk_date = row["INC_END"]

            elif row["INC_SANCTION_EXISTS"] == "N":
                at_risk_date = row['DOF']

            else:
                at_risk_date = row['INC_END']

        else:
            at_risk_date = row["INC_END"]
    
    return at_risk_date


#apply the function to the data (row by row)
df["AT_RISK_DT"] = df.apply(create_at_risk_date, axis = 1)

#adjust so that the times do not include minutes and seconds
df["AT_RISK_DT"] = pd.to_datetime(df["AT_RISK_DT"]).dt.date

#inspect the results
df[['ID_VARIABLE', 'JPR_ID',"JP_MIN", "INC_SANCTION_EXISTS", "DOS", "DOF", "AT_RISK_DT"]]



## Check for "Free Time" -- Make Sure Individuals in the Dataset Have Enough Free Time

Individuals whose **first*** offense results in an AT_RISK DATE of 2017 or greater = remove these individuals

## Populate Next DOF

In [ ]:
#sort the data
df = df.sort_values(by = ["ID_VARIABLE", "DOF"])

#set the sorted at_risk vals equal to a new df
#final_next_dof = at_risk_calc_sorted

#shift the data up by one to create the new vaariable "NEXT_DOF"
df['NEXT_DOF'] = df.groupby(['ID_VARIABLE'])['DOF'].shift(-1).dt.date

## CREATE TIME TO RECIDIVATE AND RECIDIVSM VARIABLES

In [ ]:
#subtract the next_dof and at_risk_dt variables
df['TIME_TO_RECIDIVATE'] = pd.to_datetime(df['NEXT_DOF']) - pd.to_datetime(df['AT_RISK_DT'])

#update the time to recidivate column to JUST be the number of days as an integer/float
df['TIME_TO_RECIDIVATE'] = df['TIME_TO_RECIDIVATE'].dt.days

In [ ]:
#number of days in  years
three_years_in_days = float(3) * 365.0  
five_years_in_days = float(5) * 365.0  

#JUDICIAL-PROCEEDING LEVEL RECIDIVISM
#final_next_dof["RECIDIVISM_3Y"] = final_next_dof.apply(create_recidivism_var, years = 3, axis = 1)

df["RECIDIVISM_3Y"] = np.where(
    df['TIME_TO_RECIDIVATE'] <= three_years_in_days, 1, 0)

df["RECIDIVISM_5Y"] = np.where(
    df['TIME_TO_RECIDIVATE'] <= five_years_in_days, 1, 0)

# Export The Results to CSV (MAKE SURE DATA IS AT THE DOS, ID_VARIABLE LEVEL)

In [ ]:
#Export the Results to a CSV

#double check that the 

#export the dataframe with the recidivism variables to a new dataframe
output_path = os.path.join(pa_sentencing_path, "Project", "data", "recidivsm_dataset.csv")

df.to_csv(output_path) #export the final results